In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#extrapolate the approx epidentures for Medicare Part B in 2017
F5 = pd.read_csv("F5_NHPhys_util anonymous.csv")
F5.head()

,npi,Random NPI,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_city,nppes_provider_zip,nppes_provider_state,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt
0,1811152549,NaN,PA,M,I,NASHUA,30634068,NH,Physician Assistant,Y,O,82962,Blood glucose (sugar) test performed by hand-h...,N,444.0,351,444,3.21000,5.00,3.150000,3.150000
1,1811152549,NaN,PA,M,I,NASHUA,30634068,NH,Physician Assistant,Y,O,83036,Hemoglobin A1C level,N,407.0,330,407,13.32000,22.00,13.050000,13.050000
2,1811152549,NaN,PA,M,I,NASHUA,30634068,NH,Physician Assistant,Y,O,90662,Vaccine for influenza for injection into muscle,Y,24.0,24,24,48.24125,58.25,47.277500,47.277500
3,1811152549,NaN,PA,M,I,NASHUA,30634068,NH,Physician Assistant,Y,O,95251,Ambulatory continuous glucose (sugar) includin...,N,14.0,12,14,38.31000,85.00,30.040000,29.660000
4,1811152549,NaN,PA,M,I,NASHUA,30634068,NH,Physician Assistant,Y,O,99213,Established patient office or other outpatient...,N,62.0,59,62,64.38000,178.00,46.472258,45.368871


In [2]:
#this is gonna kill my ram and is a sin, but im just gonna load in both IP OP files
in_patient = pd.read_csv("F3_IP_Sample.csv")
out_patient = pd.read_csv("F4_OP_Sample.csv")
cols_to_keep = pd.read_csv("F1_colstokeep.csv")

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,8,9,23,25,30,31,32,59,60,61,62,81,82,83,84,85,86,87,88,89,90,91,92,93,96,97,98,103,104,105,106,107,108,109,110,111,112,113,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,156,158,159,160,161,162,163,164,165,166,167,168,169,176,177,178,179,180,183,185,186,187,242,243,244,245,246) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,8,9,23,25,30,31,32,33,36,37,38,39,41,45,46,47,48,49,50,51,52,53,54,55,59,60,61,62,81,82,83,84,85,86,87,88,89,90,91,94,95,96,97,103,104,105,106,107,108,109,110,111,112,113,114,124,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,

In [3]:
print(in_patient.shape)
print(out_patient.shape)

(37959, 266)
(284894, 266)


In [4]:
cols_to_keep = pd.read_csv("F1_colstokeep.csv")
cols_to_keep = [col[0] for col in list(cols_to_keep.values)]
cols_to_keep

['DISCHARGE_KEY',
 'HOSPITAL_NME',
 'PT_AGE',
 'PT_GENDER_CDE',
 'PT_RACE_1_CDE',
 'PT_ETHNICITY_CDE',
 'PT_LANGUAGE_GROUP',
 'PT_RESIDENCE_CDE',
 'DISCHARGE_TYPE',
 'ED_FLAG',
 'OBS_FLAG',
 'LENGTH_OF_STAY',
 'ADMISSION_HOUR_NBR',
 'ADMISSION_SOURCE_CDE',
 'PT_RELATIONSHIP_CDE',
 'PRIMARY_PAY_GROUP_TXT',
 'TTL_DISCHARGE_CHG_AMT',
 'ADMITTING_DX_CDE',
 'ICD_VERSION',
 'PRINCIPAL_DX_CDE',
 'HDD_HOSPITAL_CDE',
 'FACILITY_TYPE_CDE',
 'CLAIM_FREQ',
 'DISCHARGE_HOUR_NBR',
 'PT_DISCHARGE_STATUS_CDE',
 'OTHER_DX_1_CDE',
 'OTHER_DX_2_CDE',
 'OTHER_DX_3_CDE',
 'OTHER_DX_4_CDE',
 'OTHER_DX_5_CDE',
 'OTHER_DX_6_CDE',
 'OTHER_DX_7_CDE',
 'OTHER_DX_8_CDE',
 'OTHER_DX_9_CDE',
 'OTHER_DX_10_CDE',
 'AGENCY_DRG_CDE',
 'AGENCY_MDC_CDE',
 'PRINCIPAL_PROCEDURE_CDE',
 'PRINCIPAL_DX_POA_CDE']

In [5]:
#rewrite frames
in_patient = in_patient[cols_to_keep]
out_patient = out_patient[cols_to_keep]

<h1> IP hospital rankings </h1>

<h2> Ranking by TTL_DISCHARGE_CHG_AMT </h2>

In [6]:
#first order hospitals alphabetically
hospitals_alpha = sorted(list(in_patient['HOSPITAL_NME'].unique()))
hospitals_alpha

['Alice Peck Day Memorial Hospital',
 'Androscoggin Valley Hospital',
 'Catholic Medical Center',
 'Cheshire Medical Center',
 'Concord Hospital',
 'Cottage Hospital',
 'Dartmouth Hitchcock Medical Center',
 'Elliot Hospital',
 'Exeter Hospital',
 'Franklin Regional Hospital',
 'Frisbie Memorial Hospital',
 'Huggins Hospital',
 'Lakes Region General Healthcare',
 'Littleton Regional Hospital',
 'Memorial Hospital',
 'Monadnock Community Hospital',
 'New London Hospital',
 'Parkland Medical Center',
 'Portsmouth Regional Hospital',
 'Southern New Hampshire Medical Center',
 'Speare Memorial Hospital',
 'St. Joseph Hospital',
 'Upper Connecticut Valley Hospital',
 'Valley Regional Hospital',
 'Weeks Medical Center',
 'Wentworth Douglass Hospital']

In [7]:
#most expensive first
IP_hospitals_ranked_TTL_DISCHARGE = list(in_patient.groupby("HOSPITAL_NME").describe()['TTL_DISCHARGE_CHG_AMT'].sort_values(by='mean',
                                                                                  ascending=False).index)

#define functino to get list of hospitals sorted avgmount ascending
def order_hospitals(df=in_patient,col = "HOSPITAL_NME",var = "TTL_DISCHARGE_CHG_AMT"):
    return(list(df.groupby(col).describe()[var].sort_values(by='mean',ascending=False).index))

<h2> Ranking ED Cases </h2>

In [55]:
IP_hospitals_ranked_ED_cases = order_hospitals(in_patient[in_patient['ED_FLAG'] == 1])
#its missing a hospital



<h1> Ranking Most Expensive DRG </h1>
    <h1> Ranking Second Most Expnesive DRG </h1>

In [9]:
IP_hosp_DRG = in_patient[in_patient["DISCHARGE_TYPE"] == 
          "IP"].groupby(['HOSPITAL_NME',"AGENCY_DRG_CDE"]).describe()["TTL_DISCHARGE_CHG_AMT"].sort_values(by='mean',ascending=False).reset_index()



In [27]:
#pull hospital names only twice from this sorted listed
#first instance is most expensive, second instance is second most expensive
IP_hospitals_ranked_Most_Expensive_DRG = []
IP_hospitals_ranked_Second_Most_Expensive_DRG = []

for i in range(0,IP_hosp_DRG.shape[0]):
    #add first hospital name
    hosp_name = IP_hosp_DRG['HOSPITAL_NME'][i]
    if hosp_name not in IP_hospitals_ranked_Most_Expensive_DRG:
        IP_hospitals_ranked_Most_Expensive_DRG.append(hosp_name)
    #condition if name in most expensive
    else:
        if hosp_name not in IP_hospitals_ranked_Second_Most_Expensive_DRG:
            IP_hospitals_ranked_Second_Most_Expensive_DRG.append(hosp_name)
                


<h1> Ranking by Discharge to Medicare </h1>

In [32]:
IP_hospitals_ranked_Discharge_Medicare = order_hospitals(in_patient[in_patient['PRIMARY_PAY_GROUP_TXT'] == "Medicare"])
IP_hospitals_ranked_Discharge_Medicare



['Portsmouth Regional Hospital',
 'Dartmouth Hitchcock Medical Center',
 'Catholic Medical Center',
 'Lakes Region General Healthcare',
 'Wentworth Douglass Hospital',
 'Concord Hospital',
 'Parkland Medical Center',
 'Exeter Hospital',
 'Cheshire Medical Center',
 'St. Joseph Hospital',
 'Southern New Hampshire Medical Center',
 'Alice Peck Day Memorial Hospital',
 'Elliot Hospital',
 'Frisbie Memorial Hospital',
 'Franklin Regional Hospital',
 'Littleton Regional Hospital',
 'Speare Memorial Hospital',
 'Cottage Hospital',
 'Memorial Hospital',
 'Huggins Hospital',
 'Weeks Medical Center',
 'Androscoggin Valley Hospital',
 'Monadnock Community Hospital',
 'New London Hospital',
 'Upper Connecticut Valley Hospital',
 'Valley Regional Hospital']

<h1> Ranking by Discharge to Medicaid </h1>

In [54]:
IP_hospitals_ranked_Discharge_Medicaid = order_hospitals(in_patient[in_patient['PRIMARY_PAY_GROUP_TXT'] == "Medicaid"])
len(IP_hospitals_ranked_Discharge_Medicaid)


25

<h1> Ranking by Discharge to Commercial Insurance </h1>

In [34]:
IP_hospitals_ranked_Discharge_Commercial = order_hospitals(in_patient[in_patient['PRIMARY_PAY_GROUP_TXT'] == "Commercial"])
IP_hospitals_ranked_Discharge_Commercial

['Portsmouth Regional Hospital',
 'Dartmouth Hitchcock Medical Center',
 'Catholic Medical Center',
 'Lakes Region General Healthcare',
 'Parkland Medical Center',
 'Concord Hospital',
 'Exeter Hospital',
 'Wentworth Douglass Hospital',
 'St. Joseph Hospital',
 'Huggins Hospital',
 'Franklin Regional Hospital',
 'Southern New Hampshire Medical Center',
 'Cottage Hospital',
 'Frisbie Memorial Hospital',
 'New London Hospital',
 'Alice Peck Day Memorial Hospital',
 'Cheshire Medical Center',
 'Elliot Hospital',
 'Weeks Medical Center',
 'Upper Connecticut Valley Hospital',
 'Littleton Regional Hospital',
 'Androscoggin Valley Hospital',
 'Speare Memorial Hospital',
 'Valley Regional Hospital',
 'Memorial Hospital',
 'Monadnock Community Hospital']

<h1> Ranking by Average billed ICD10 </h1>

In [35]:
IP_hospitals_ranked_billed_ICD10 = order_hospitals(in_patient[in_patient['ICD_VERSION'] == "ICD10"])
IP_hospitals_ranked_billed_ICD10

['Portsmouth Regional Hospital',
 'Dartmouth Hitchcock Medical Center',
 'Catholic Medical Center',
 'Lakes Region General Healthcare',
 'Parkland Medical Center',
 'Concord Hospital',
 'Wentworth Douglass Hospital',
 'Exeter Hospital',
 'St. Joseph Hospital',
 'Cheshire Medical Center',
 'Frisbie Memorial Hospital',
 'Southern New Hampshire Medical Center',
 'Elliot Hospital',
 'Alice Peck Day Memorial Hospital',
 'Cottage Hospital',
 'Weeks Medical Center',
 'Franklin Regional Hospital',
 'Littleton Regional Hospital',
 'Speare Memorial Hospital',
 'New London Hospital',
 'Memorial Hospital',
 'Monadnock Community Hospital',
 'Androscoggin Valley Hospital',
 'Upper Connecticut Valley Hospital',
 'Valley Regional Hospital']

<h1> Hospital Rankings by aforementioned ranking schemes </h1>

In [57]:
IP_hospitals_ranked = pd.DataFrame()
IP_hospitals_ranked['Hospitals Alpha'] = hospitals_alpha
IP_hospitals_ranked['Avg Total Discharge'] = IP_hospitals_ranked_TTL_DISCHARGE
IP_hospitals_ranked['Avg Discharge ED Cases'] = pd.Series(IP_hospitals_ranked_ED_cases,index=None)
IP_hospitals_ranked['Avg Discharge Most Epxpensive DRG'] = IP_hospitals_ranked_Most_Expensive_DRG
IP_hospitals_ranked['Avg Discharge Second Most Expensive DRG'] = IP_hospitals_ranked_Second_Most_Expensive_DRG
IP_hospitals_ranked['Avg Discharge to Medicare'] = IP_hospitals_ranked_Discharge_Medicare
IP_hospitals_ranked['Avg Discharge to Medicaid'] = pd.Series(IP_hospitals_ranked_Discharge_Medicaid,index=None)
IP_hospitals_ranked['Avg Discharge to Commercial Insurace'] = IP_hospitals_ranked_Discharge_Commercial
IP_hospitals_ranked['Avg Billed ICD10'] = pd.Series(IP_hospitals_ranked_billed_ICD10,index=None)

In [59]:
#define function that returns ranked order on the condition that ranking is for hospitals alpha
def order_hospitals_ranked(df,hospitals,col):
    ranking = []
    for hospital in df[hospitals]:
        #get index equal to hospital
        index = df[col] == hospital
        #in case there are none
        if sum(index) == 0:
            ranking.append(np.nan)
        #get length of index
        len_index = len(index)
        #traverse index
        for i in range(0,len_index):
            if index[i] == True:
                ranking.append(i)
            else:
                continue
        #adjust ranking, because of edge cases
        #if np.nansum(ranking) == 0:
            #find zero
         #   len_ranking = len(ranking)
            #traverse new ranking
          #  for j in range(0,len_ranking):
           #     if ranking[j] == 0:
                    #put index as value
            #        ranking[j] = j
    return(ranking)
order_hospitals_ranked(df = IP_hospitals_ranked,hospitals = 'Hospitals Alpha',col = 'Avg Discharge Second Most Expensive DRG')
#FINALLY!

[21,
 17,
 2,
 11,
 3,
 14,
 0,
 12,
 5,
 23,
 10,
 22,
 4,
 13,
 18,
 24,
 20,
 8,
 1,
 7,
 15,
 9,
 25,
 16,
 19,
 6]

In [63]:
for column in list(IP_hospitals_ranked.columns)[1:]:
    #use abover function
    IP_hospitals_ranked[column] = order_hospitals_ranked(df = IP_hospitals_ranked,hospitals = 'Hospitals Alpha',
                                                      col = column)

In [65]:
IP_hospitals_ranked

,Hospitals Alpha,Avg Total Discharge,Avg Discharge ED Cases,Avg Discharge Most Epxpensive DRG,Avg Discharge Second Most Expensive DRG,Avg Discharge to Medicare,Avg Discharge to Medicaid,Avg Discharge to Commercial Insurace,Avg Billed ICD10
0,Alice Peck Day Memorial Hospital,14,23.0,19,21,11,20.0,15,13.0
1,Androscoggin Valley Hospital,22,19.0,20,17,21,18.0,21,22.0
2,Catholic Medical Center,2,3.0,3,2,2,7.0,2,2.0
3,Cheshire Medical Center,9,17.0,5,11,8,6.0,16,9.0
4,Concord Hospital,5,5.0,0,3,5,2.0,5,5.0
5,Cottage Hospital,15,9.0,14,14,17,10.0,12,14.0
6,Dartmouth Hitchcock Medical Center,1,1.0,4,0,1,1.0,1,1.0
7,Elliot Hospital,12,8.0,12,12,12,12.0,17,12.0
8,Exeter Hospital,7,15.0,7,5,7,8.0,6,7.0
9,Franklin Regional Hospital,13,6.0,23,23,14,17.0,10,16.0


<h1> New Ranking, sum acoss, average, if give NA zero for missing </h1>

In [66]:
np.nansum(IP_hospitals_ranked.iloc[3,1:].values)
weight_sum_avg_ranking = []
for i in range(0, IP_hospitals_ranked.shape[0]):
    #compute ranke
    new_rank = np.nansum(IP_hospitals_ranked.iloc[i][1:].values) / 8
    #append rank
    weight_sum_avg_ranking.append(new_rank)
weight_sum_avg_ranking

[17.0,
 20.0,
 2.875,
 10.125,
 3.75,
 13.125,
 1.25,
 12.125,
 7.75,
 15.25,
 11.0,
 16.375,
 3.125,
 17.25,
 20.25,
 22.125,
 16.375,
 5.625,
 0.5,
 10.875,
 18.25,
 8.875,
 19.375,
 18.625,
 16.25,
 7.5]

<h1> New Ranking, TTL_DISCHARGE_CHG_AMT and ED cases weighted by 2 </h1>
<h1> New Ranking, Medicare, Medicaid, Commercial Insurance, weighted by 1.5 </h1>
<h1> New Ranking, Most Expensive Drg, Second Most Expensive DRG, Avg Billed ICD10, weighted by 1 </h1>

In [67]:
new_weighting_scheme = []
for i in range(0, IP_hospitals_ranked.shape[0]):
    #store values
    values = list(IP_hospitals_ranked.iloc[i][1:9].values)
    #adjust values
    values[0] = values[0]*2
    values[1] = values[1]*2
    values[4] = values[4]*1.5
    values[5] = values[5]*1.5
    values[6] = values[6]*1.5
    #get rank of new values
    new_rank = np.nansum(values) / 8
    #append rank
    new_weighting_scheme.append(new_rank)

In [68]:
#addd the new ranking scores
IP_hospitals_ranked["Equal_weight_Score"] = weight_sum_avg_ranking
IP_hospitals_ranked['Adjusted_weight_score'] = new_weighting_scheme

<h1> Final Rankings, note it starts at zero, if want to start at 1, and 1 to all rankings </h1>

In [69]:
IP_hospitals_ranked

,Hospitals Alpha,Avg Total Discharge,Avg Discharge ED Cases,Avg Discharge Most Epxpensive DRG,Avg Discharge Second Most Expensive DRG,Avg Discharge to Medicare,Avg Discharge to Medicaid,Avg Discharge to Commercial Insurace,Avg Billed ICD10,Equal_weight_Score,Adjusted_weight_score
0,Alice Peck Day Memorial Hospital,14,23.0,19,21,11,20.0,15,13.0,17.000,24.5000
1,Androscoggin Valley Hospital,22,19.0,20,17,21,18.0,21,22.0,20.000,28.8750
2,Catholic Medical Center,2,3.0,3,2,2,7.0,2,2.0,2.875,4.1875
3,Cheshire Medical Center,9,17.0,5,11,8,6.0,16,9.0,10.125,15.2500
4,Concord Hospital,5,5.0,0,3,5,2.0,5,5.0,3.750,5.7500
5,Cottage Hospital,15,9.0,14,14,17,10.0,12,14.0,13.125,18.5625
6,Dartmouth Hitchcock Medical Center,1,1.0,4,0,1,1.0,1,1.0,1.250,1.6875
7,Elliot Hospital,12,8.0,12,12,12,12.0,17,12.0,12.125,17.1875
8,Exeter Hospital,7,15.0,7,5,7,8.0,6,7.0,7.750,11.8125
9,Franklin Regional Hospital,13,6.0,23,23,14,17.0,10,16.0,15.250,20.1875


<h1> OUTPATIENT RANKINGS </h1>

In [7]:
#most expensive first
OP_hospitals_ranked_TTL_DISCHARGE = list(out_patient.groupby("HOSPITAL_NME").describe()['TTL_DISCHARGE_CHG_AMT'].sort_values(by='mean',
                                                                                  ascending=False).index)

#most expensive first
IP_hospitals_ranked_TTL_DISCHARGE = list(in_patient.groupby("HOSPITAL_NME").describe()['TTL_DISCHARGE_CHG_AMT'].sort_values(by='mean',
                                                                                  ascending=False).index)

#define functino to get list of hospitals sorted avgmount ascending
def order_hospitals(df=in_patient,col = "HOSPITAL_NME",var = "TTL_DISCHARGE_CHG_AMT"):
    return(list(df.groupby(col).describe()[var].sort_values(by='mean',ascending=False).index))
OP_hospitals_ranked_TTL_DISCHARGE

['Parkland Medical Center',
 'St. Joseph Hospital',
 'Dartmouth Hitchcock Medical Center',
 'Alice Peck Day Memorial Hospital',
 'Portsmouth Regional Hospital',
 'Upper Connecticut Valley Hospital',
 'Speare Memorial Hospital',
 'Memorial Hospital',
 'Exeter Hospital',
 'Frisbie Memorial Hospital',
 'Wentworth Douglass Hospital',
 'Cottage Hospital',
 'Catholic Medical Center',
 'Concord Hospital',
 'Littleton Regional Hospital',
 'Cheshire Medical Center',
 'Huggins Hospital',
 'Elliot Hospital',
 'Valley Regional Hospital',
 'Monadnock Community Hospital',
 'New London Hospital',
 'Southern New Hampshire Medical Center',
 'Lakes Region General Healthcare',
 'Androscoggin Valley Hospital',
 'Franklin Regional Hospital',
 'Weeks Medical Center']

<h1> Ranking ED cases </h1>

In [8]:
#subset ED_flag ==1
OP_hospitals_ranked_ED_cases = order_hospitals(out_patient[out_patient['ED_FLAG'] == 1])
OP_hospitals_ranked_ED_cases

['Parkland Medical Center',
 'Portsmouth Regional Hospital',
 'Wentworth Douglass Hospital',
 'Catholic Medical Center',
 'Lakes Region General Healthcare',
 'Dartmouth Hitchcock Medical Center',
 'Cheshire Medical Center',
 'Elliot Hospital',
 'St. Joseph Hospital',
 'Upper Connecticut Valley Hospital',
 'Concord Hospital',
 'Franklin Regional Hospital',
 'Speare Memorial Hospital',
 'Southern New Hampshire Medical Center',
 'Exeter Hospital',
 'Monadnock Community Hospital',
 'Huggins Hospital',
 'Androscoggin Valley Hospital',
 'Valley Regional Hospital',
 'New London Hospital',
 'Cottage Hospital',
 'Frisbie Memorial Hospital',
 'Weeks Medical Center',
 'Littleton Regional Hospital',
 'Memorial Hospital',
 'Alice Peck Day Memorial Hospital']

<h1> Ranking by Discharge to Medicare </h1>

In [9]:
OP_hospitals_ranked_Discharge_Medicare = order_hospitals(out_patient[out_patient['PRIMARY_PAY_GROUP_TXT'] == "Medicare"])
OP_hospitals_ranked_Discharge_Medicare

['St. Joseph Hospital',
 'Alice Peck Day Memorial Hospital',
 'Parkland Medical Center',
 'Dartmouth Hitchcock Medical Center',
 'Upper Connecticut Valley Hospital',
 'Memorial Hospital',
 'Speare Memorial Hospital',
 'Portsmouth Regional Hospital',
 'Exeter Hospital',
 'Cottage Hospital',
 'Frisbie Memorial Hospital',
 'Wentworth Douglass Hospital',
 'Littleton Regional Hospital',
 'Catholic Medical Center',
 'Valley Regional Hospital',
 'Huggins Hospital',
 'New London Hospital',
 'Southern New Hampshire Medical Center',
 'Concord Hospital',
 'Monadnock Community Hospital',
 'Weeks Medical Center',
 'Elliot Hospital',
 'Cheshire Medical Center',
 'Androscoggin Valley Hospital',
 'Lakes Region General Healthcare',
 'Franklin Regional Hospital']

<h1> Ranking by Discharge to Medicaid </h1>

In [10]:
OP_hospitals_ranked_Discharge_Medicaid = order_hospitals(out_patient[out_patient['PRIMARY_PAY_GROUP_TXT'] == "Medicaid"])
OP_hospitals_ranked_Discharge_Medicaid

['Parkland Medical Center',
 'Portsmouth Regional Hospital',
 'Dartmouth Hitchcock Medical Center',
 'Wentworth Douglass Hospital',
 'Cheshire Medical Center',
 'St. Joseph Hospital',
 'Upper Connecticut Valley Hospital',
 'Speare Memorial Hospital',
 'Franklin Regional Hospital',
 'Androscoggin Valley Hospital',
 'Catholic Medical Center',
 'Concord Hospital',
 'Lakes Region General Healthcare',
 'Exeter Hospital',
 'Valley Regional Hospital',
 'Memorial Hospital',
 'Elliot Hospital',
 'Cottage Hospital',
 'Huggins Hospital',
 'Frisbie Memorial Hospital',
 'Alice Peck Day Memorial Hospital',
 'Monadnock Community Hospital',
 'Littleton Regional Hospital',
 'Southern New Hampshire Medical Center',
 'New London Hospital',
 'Weeks Medical Center']

<h1> Ranking by Discharge to Commercial Insurance </h1>

In [11]:
OP_hospitals_ranked_Discharge_Commercial = order_hospitals(out_patient[out_patient['PRIMARY_PAY_GROUP_TXT'] == "Commercial"])
OP_hospitals_ranked_Discharge_Commercial


['Parkland Medical Center',
 'St. Joseph Hospital',
 'Alice Peck Day Memorial Hospital',
 'Dartmouth Hitchcock Medical Center',
 'Portsmouth Regional Hospital',
 'Speare Memorial Hospital',
 'Memorial Hospital',
 'Exeter Hospital',
 'Cheshire Medical Center',
 'Upper Connecticut Valley Hospital',
 'Frisbie Memorial Hospital',
 'Catholic Medical Center',
 'Concord Hospital',
 'Wentworth Douglass Hospital',
 'Cottage Hospital',
 'Elliot Hospital',
 'Littleton Regional Hospital',
 'Huggins Hospital',
 'Monadnock Community Hospital',
 'Lakes Region General Healthcare',
 'Southern New Hampshire Medical Center',
 'Valley Regional Hospital',
 'New London Hospital',
 'Franklin Regional Hospital',
 'Androscoggin Valley Hospital',
 'Weeks Medical Center']

<h1> Ranking by Average billed ICD10 </h1>

In [12]:
OP_hospitals_ranked_billed_ICD10 = order_hospitals(out_patient[out_patient['ICD_VERSION'] == "ICD10"])
OP_hospitals_ranked_billed_ICD10

['Parkland Medical Center',
 'St. Joseph Hospital',
 'Alice Peck Day Memorial Hospital',
 'Portsmouth Regional Hospital',
 'Speare Memorial Hospital',
 'Upper Connecticut Valley Hospital',
 'Dartmouth Hitchcock Medical Center',
 'Memorial Hospital',
 'Exeter Hospital',
 'Wentworth Douglass Hospital',
 'Frisbie Memorial Hospital',
 'Catholic Medical Center',
 'Cheshire Medical Center',
 'Concord Hospital',
 'Cottage Hospital',
 'Littleton Regional Hospital',
 'Huggins Hospital',
 'Elliot Hospital',
 'Valley Regional Hospital',
 'Southern New Hampshire Medical Center',
 'New London Hospital',
 'Monadnock Community Hospital',
 'Lakes Region General Healthcare',
 'Franklin Regional Hospital',
 'Androscoggin Valley Hospital',
 'Weeks Medical Center']

<h1> OP Hospital Rankings by aforementioned ranking schemes </h1>

In [13]:
OP_hospitals_ranked = pd.DataFrame()
OP_hospitals_ranked['Hospitals Alpha'] = hospitals_alpha
OP_hospitals_ranked['Avg Total Discharge'] = OP_hospitals_ranked_TTL_DISCHARGE
OP_hospitals_ranked['Avg Discharge ED Cases'] = OP_hospitals_ranked_ED_cases
OP_hospitals_ranked['Avg Discharge to Medicare'] = OP_hospitals_ranked_Discharge_Medicare
OP_hospitals_ranked['Avg Discharge to Medicaid'] = OP_hospitals_ranked_Discharge_Medicaid
OP_hospitals_ranked['Avg Discharge to Commercial Insurace'] = OP_hospitals_ranked_Discharge_Commercial
OP_hospitals_ranked['Avg Billed ICD10'] = OP_hospitals_ranked_billed_ICD10

In [14]:
OP_hospitals_ranked.head()

,Hospitals Alpha,Avg Total Discharge,Avg Discharge ED Cases,Avg Discharge to Medicare,Avg Discharge to Medicaid,Avg Discharge to Commercial Insurace,Avg Billed ICD10
0,Alice Peck Day Memorial Hospital,Parkland Medical Center,Parkland Medical Center,St. Joseph Hospital,Parkland Medical Center,Parkland Medical Center,Parkland Medical Center
1,Androscoggin Valley Hospital,St. Joseph Hospital,Portsmouth Regional Hospital,Alice Peck Day Memorial Hospital,Portsmouth Regional Hospital,St. Joseph Hospital,St. Joseph Hospital
2,Catholic Medical Center,Dartmouth Hitchcock Medical Center,Wentworth Douglass Hospital,Parkland Medical Center,Dartmouth Hitchcock Medical Center,Alice Peck Day Memorial Hospital,Alice Peck Day Memorial Hospital
3,Cheshire Medical Center,Alice Peck Day Memorial Hospital,Catholic Medical Center,Dartmouth Hitchcock Medical Center,Wentworth Douglass Hospital,Dartmouth Hitchcock Medical Center,Portsmouth Regional Hospital
4,Concord Hospital,Portsmouth Regional Hospital,Lakes Region General Healthcare,Upper Connecticut Valley Hospital,Cheshire Medical Center,Portsmouth Regional Hospital,Speare Memorial Hospital


In [16]:
#define function that returns ranked order on the condition that ranking is for hospitals alpha
def order_hospitals_ranked(df,hospitals,col):
    ranking = []
    for hospital in df[hospitals]:
        #get index equal to hospital
        index = df[col] == hospital
        #in case there are none
        if sum(index) == 0:
            ranking.append(np.nan)
        #get length of index
        len_index = len(index)
        #traverse index
        for i in range(0,len_index):
            if index[i] == True:
                ranking.append(i)
            else:
                continue
        #adjust ranking, because of edge cases
        #if np.nansum(ranking) == 0:
            #find zero
         #   len_ranking = len(ranking)
            #traverse new ranking
          #  for j in range(0,len_ranking):
           #     if ranking[j] == 0:
                    #put index as value
            #        ranking[j] = j
    return(ranking)
order_hospitals_ranked(df = OP_hospitals_ranked,hospitals = 'Hospitals Alpha',col = 'Avg Total Discharge')
#FINALLY!
#use new function to overwrite columns

[3,
 23,
 12,
 15,
 13,
 11,
 2,
 17,
 8,
 24,
 9,
 16,
 22,
 14,
 7,
 19,
 20,
 0,
 4,
 21,
 6,
 1,
 5,
 18,
 25,
 10]

In [17]:
for column in list(OP_hospitals_ranked.columns)[1:]:
    #use abover function
    OP_hospitals_ranked[column] = order_hospitals_ranked(df = OP_hospitals_ranked,hospitals = 'Hospitals Alpha',
                                                      col = column)

In [18]:
OP_hospitals_ranked.head()

,Hospitals Alpha,Avg Total Discharge,Avg Discharge ED Cases,Avg Discharge to Medicare,Avg Discharge to Medicaid,Avg Discharge to Commercial Insurace,Avg Billed ICD10
0,Alice Peck Day Memorial Hospital,3,25,1,20,2,2
1,Androscoggin Valley Hospital,23,17,23,9,24,24
2,Catholic Medical Center,12,3,13,10,11,11
3,Cheshire Medical Center,15,6,22,4,8,12
4,Concord Hospital,13,10,18,11,12,13


<h1> New Ranking, sum acoss, average, if give NA zero for missing </h1>

In [19]:
weight_sum_avg_ranking = []
for i in range(0, OP_hospitals_ranked.shape[0]):
    #compute ranke
    new_rank = np.nansum(OP_hospitals_ranked.iloc[i][1:].values) / 6
    #append rank
    weight_sum_avg_ranking.append(new_rank)
weight_sum_avg_ranking

[8.833333333333334,
 20.0,
 10.0,
 11.166666666666666,
 12.833333333333334,
 14.166666666666666,
 3.5,
 15.5,
 9.666666666666666,
 19.0,
 13.166666666666666,
 16.333333333333332,
 17.166666666666668,
 17.0,
 10.666666666666666,
 18.833333333333332,
 20.166666666666668,
 0.3333333333333333,
 3.3333333333333335,
 18.833333333333332,
 6.666666666666667,
 2.6666666666666665,
 6.333333333333333,
 17.166666666666668,
 23.666666666666668,
 8.0]

<h1> New Ranking, TTL_DISCHARGE_CHG_AMT and ED cases weighted by 2 </h1>
<h1> New Ranking, Medicare, Medicaid, Commercial Insurance, weighted by 1.5 </h1>
<h1> New Ranking, Avg Billed ICD10, weighted by 1 </h1>

In [22]:
new_weighting_scheme = []
for i in range(0, OP_hospitals_ranked.shape[0]):
    #store values
    values = list(OP_hospitals_ranked.iloc[i][1:6].values)
    #adjust values
    values[0] = values[0]*2
    values[1] = values[1]*2
    values[2] = values[2]*1.5
    values[3] = values[3]*1.5
    values[4] = values[4]*1.5
    #get rank of new values
    new_rank = np.nansum(values) / 6
    #append rank
    new_weighting_scheme.append(new_rank)

In [23]:
#addd the new ranking scores
OP_hospitals_ranked["Equal_weight_Score"] = weight_sum_avg_ranking
OP_hospitals_ranked['Adjusted_weight_score'] = new_weighting_scheme

<h1> Final Rankings, note it starts at zero, if want to start at 1, and 1 to all rankings </h1>

In [24]:
OP_hospitals_ranked

,Hospitals Alpha,Avg Total Discharge,Avg Discharge ED Cases,Avg Discharge to Medicare,Avg Discharge to Medicaid,Avg Discharge to Commercial Insurace,Avg Billed ICD10,Equal_weight_Score,Adjusted_weight_score
0,Alice Peck Day Memorial Hospital,3,25,1,20,2,2,8.833333,15.083333
1,Androscoggin Valley Hospital,23,17,23,9,24,24,20.000000,27.333333
2,Catholic Medical Center,12,3,13,10,11,11,10.000000,13.500000
3,Cheshire Medical Center,15,6,22,4,8,12,11.166667,15.500000
4,Concord Hospital,13,10,18,11,12,13,12.833333,17.916667
5,Cottage Hospital,11,20,9,17,14,14,14.166667,20.333333
6,Dartmouth Hitchcock Medical Center,2,5,3,2,3,6,3.500000,4.333333
7,Elliot Hospital,17,7,21,16,15,17,15.500000,21.000000
8,Exeter Hospital,8,14,8,13,7,8,9.666667,14.333333
9,Franklin Regional Hospital,24,11,25,8,23,23,19.000000,25.666667
